# I. Overview:

* This ipython notebook utilizes the __batch_alerts module__ to generate near real time alerts for feeder level outages
* Inputs:
    * Daily raw data (edna, ami, scada) from FPL (in csv form)
    * Meta data on the feeders (for ami anomaly transformation)
    * Congiguration files
    * pkl clf model
* Outputs:
    * dictionary of alerter:
        * key = feeder
        * value = alerter (collection of alerts) of a feeder 

# II. Libraries:

In [6]:
from autogrid.pam.anomaly import anomaly
from autogrid.pam.alert import alert 
from autogrid.pam.batch_alerts.batch_alerts import BatchAlerts

import cPickle as pickle
import pandas as pd
import yaml

# III. Functions:

In [7]:
# function #1:
def combine_preds_df(alerts_dict):
    '''
    PURPOSE:
        - combine prediction table of different fdrs into one df
    INPUT:
        - alerts_dict (dict; keys: fdr id; values; alerter object)
    OUTPUT:
        - preds_df (df of combined preds with sigantures)
    '''
    first = True
    for fdr in alerts_dict.keys():
        if alerts_dict[fdr].preds.empty:
            continue
        else:
            loc_preds_table = alerts_dict[fdr].preds
            loc_preds_table = pd.concat([loc_preds_table, alerts_dict[fdr].X], axis=1)            
        if first:
            preds_table = loc_preds_table
            first = False
        else:
            preds_table = pd.concat([preds_table, loc_preds_table])
            
    return preds_table

# IV. Decalre Variables:

In [8]:
# i. model name:
m_id, n_jobs = 'model_3_0', 7
# ii. dates:
start_date, end_date =\
    pd.datetime(2017, 6, 13).date(), pd.datetime(2017, 6, 14).date()
# iii. pwds:
raw_data_pwd =\
    '/Users/meghavalsan/Dropbox (AutoGrid)/sqlfiles/'
meta_pwd =\
    '/Users/meghavalsan/megha_github/model3files/feedermeta/'
model_pwd =\
    '/Users/meghavalsan/megha_github/model3files/modelpkl/model_v_3_0.pkl'
config_pwd =\
    '/Users/meghavalsan/megha_github/model3files/config/'
dataset_config_pwd =\
    config_pwd + 'pam_3_0_dataset.yaml'
anomaly_map_pwd =\
    config_pwd + 'pam_3_0_anoms_anomaly_map.yaml'
alert_config_pwd =\
    config_pwd + 'pam_3_0_alert.yaml'

# V. Alerts Generation:

In [9]:
anomalyfile=yaml.load(open(anomaly_map_pwd, 'rb'))
anomslist=anomalyfile.keys()
print anomslist
# i. initiate class:
june_13_14_2017_alerts = BatchAlerts(
    n_jobs, start_date, end_date,
    raw_data_pwd, meta_pwd,anomslist)

['ZERO_VOLTAGE_V3', 'AFS_GROUND_ALARM', 'INTELI_PH_ALARM', 'LG_PD_10', 'ZERO_POWER_V3', 'FDRHD_DE_ENERGIZED']


In [10]:
# ii. load data:
june_13_14_2017_alerts.load_data()

/Users/meghavalsan/megha_github/pammodel3env/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2881: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
# iii. process anomalies:
june_13_14_2017_alerts.generates_anomalies()

Launching 7 jobs for edna data!


[Parallel(n_jobs=7)]: Done   1 out of   7 | elapsed:  5.4min remaining: 32.6min
[Parallel(n_jobs=7)]: Done   7 out of   7 | elapsed: 29.1min finished


In [14]:
june_13_14_2017_alerts.anoms_df1

,Anomaly,DeviceId,DevicePh,DeviceType,Feeder,Signal,Time
0,ZERO_POWER_V3,2W75,-,PHASER,400433,HILLCREST.FDR.400433_2W75.MW,2017-06-13 21:36:23+00:00
1,ZERO_VOLTAGE_V3,2W75,A,PHASER,400433,HILLCREST.FDR.400433_2W75.V.A_PH,2017-06-13 21:36:23+00:00
2,ZERO_VOLTAGE_V3,2W75,B,PHASER,400433,HILLCREST.FDR.400433_2W75.V.B_PH,2017-06-13 21:36:23+00:00
3,ZERO_VOLTAGE_V3,2W75,C,PHASER,400433,HILLCREST.FDR.400433_2W75.V.C_PH,2017-06-13 21:36:23+00:00
0,ZERO_POWER_V3,2W53,C,PHASER,800533,RIVERSIDE.FDR.800533_2W53.MW.C_PH,2017-06-14 06:32:31+00:00
0,ZERO_POWER_V3,2W24,-,PHASER,201736,SYKES_CREEK.FDR.201736_2W24.MW,2017-06-14 00:28:03+00:00
1,ZERO_VOLTAGE_V3,2W24,A,PHASER,201736,SYKES_CREEK.FDR.201736_2W24.V.A_PH,2017-06-14 00:28:03+00:00
2,ZERO_VOLTAGE_V3,2W24,B,PHASER,201736,SYKES_CREEK.FDR.201736_2W24.V.B_PH,2017-06-14 00:28:03+00:00
3,ZERO_VOLTAGE_V3,2W24,C,PHASER,201736,SYKES_CREEK.FDR.201736_2W24.V.C_PH,2017-06-14 00:28:03+00:00
0,AFS_GROUND_ALARM,A13772,-,AFS,205361,GENEVA.205361.AFS.A13772.GROUND,2017-06-13 12:40:11+00:00


In [12]:
june_13_14_2017_alerts.anoms_df1=june_13_14_2017_alerts.anoms_df.loc[june_13_14_2017_alerts.anoms_df['Time']>=pd.datetime(2017,6,13)]

In [13]:
june_13_14_2017_alerts.anoms_df1.shape

(316, 7)

In [13]:
# iv. generate alerts:
june_13_14_2017_alerts.generates_alerts(
        m_id, model_pwd,
        dataset_config_pwd,
        anomaly_map_pwd, alert_config_pwd
    )

/Users/meghavalsan/megha_github/pammodel3env/lib/python2.7/site-packages/pandas/core/generic.py:2387: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/Users/meghavalsan/megha_github/pammodel3env/lib/python2.7/site-packages/pandas/core/generic.py:2387: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/Users/meghavalsan/megha_github/pammodel3env/lib/python2.7/site-packages/pandas/core/generic.py:2387: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [ ]:
# v. pkl results:
pickle.dump(june_13_14_2017_alerts.alerts_dict, open('jun_13_14_2017_alerts_dict_m_3_0.pkl', 'wb'))
pickle.dump(june_13_14_2017_alerts.anoms_df, open('jun_13_14_2017_anoms_df_m_3_0.pkl', 'wb'))

# VI. Alerts EDA:

## June 13 ~ 14, 2017:

In [40]:
june_13_14_2017_preds_df =\
    combine_preds_df(june_13_14_2017_alerts.alerts_dict)

In [41]:
june_13_14_2017_preds_df.shape

(859, 11)

In [42]:
june_13_14_2017_preds_df['FEEDER'].unique().shape

(602,)

In [48]:
new_fr1=pd.DataFrame()
for fdr in june_13_14_2017_preds_df['FEEDER'].unique():
    loc_df =\
        june_13_14_2017_preds_df.loc[june_13_14_2017_preds_df['FEEDER']==fdr]
    if loc_df.shape[0] > 1:
        for d in loc_df['TIMESTAMP'].dt.date.unique():
            ld_df=loc_df.loc[loc_df['TIMESTAMP'].dt.date==d]
            if ld_df.shape[0] > 1:
                new_fr1=new_fr1.append(ld_df.tail(1))
            else:
                new_fr1=new_fr1.append(ld_df)
            
    else:
        new_fr1=new_fr1.append(loc_df)

In [66]:
new_fr1=new_fr1.reset_index()
new_fr1.drop(['index'],axis=1,inplace=True)
new_fr1

,FEEDER,TIMESTAMP,PROB,ALERT,TIER,LG_PD_10_24,ZERO_VOLTAGE_V3_24,AFS_GROUND_ALARM_24,FDRHD_DE_ENERGIZED_24,ZERO_POWER_V3_24,INTELI_PH_ALARM_24
0,500232,2017-06-17 12:03:00+00:00,0.059397,Yellow,0,1,0,0,0,0,0
1,500236,2017-06-17 12:02:00+00:00,0.059565,Yellow,0,1,0,0,1,1,0
2,500235,2017-06-17 12:03:00+00:00,0.059397,Yellow,0,1,0,0,0,0,0
3,500333,2017-06-17 11:08:00+00:00,0.059565,Yellow,0,1,3,0,1,1,0
4,500239,2017-06-17 12:03:00+00:00,0.059397,Yellow,0,1,0,0,0,0,0
5,500334,2017-06-17 11:09:00+00:00,0.059397,Yellow,0,1,0,0,0,0,0
6,706166,2017-06-16 20:21:00+00:00,0.060957,Yellow,0,2,0,0,0,0,0
7,706166,2017-06-17 16:57:00+00:00,0.060957,Yellow,0,2,0,0,1,0,0
8,706167,2017-06-16 20:21:00+00:00,0.059397,Yellow,0,1,0,0,0,0,0
9,706167,2017-06-17 16:57:00+00:00,0.060957,Yellow,0,2,0,0,0,0,0


In [14]:
june_13_14_2017_preds_df.loc[june_13_14_2017_preds_df['FEEDER']=='504763']

,FEEDER,TIMESTAMP,PROB,ALERT,TIER,LG_PD_10_24,ZERO_VOLTAGE_V3_24,AFS_GROUND_ALARM_24,FDRHD_DE_ENERGIZED_24,ZERO_POWER_V3_24,INTELI_PH_ALARM_24
0,504763,2017-06-13 12:03:00+00:00,0.059397,Yellow,0,1,0,0,0,0,0
1,504763,2017-06-14 16:38:00+00:00,0.059397,Yellow,0,1,0,0,0,0,0
2,504763,2017-06-14 16:55:00+00:00,0.060957,Yellow,0,2,0,0,0,0,0


## June 15 ~ 16, 2017:

## June 17 ~ 18, 2017:

## June 19 ~ 20, 2017: